In [3]:
# Amtrak (Amtraker v3) Live Train Tracking — Notebook-ready client
# -------------------------------------------------------------------
# API base: https://api-v3.amtraker.com
# Key endpoints (v3):
#   /v3/trains                  → dict: trainNum -> list[Train]
#   /v3/trains/{trainIdOrNum}   → dict for a single train number or "trainId" (e.g., "5-9")
#   /v3/stations                → dict: stationId -> StationMeta
#   /v3/stations/{stationId}    → dict for a single station's meta
#   /v3/stale                   → { avgLastUpdate, activeTrains, stale }   (feed health)
#   /v3/oembed?url=...          → oEmbed data for Amtraker URLs
# -------------------------------------------------------------------

from __future__ import annotations

import time
import typing as t
import requests

try:
    import pandas as pd  # optional
    _HAS_PANDAS = True
except Exception:
    _HAS_PANDAS = False

AMTRAK_V3_BASE_URL = "https://api-v3.amtraker.com"

class AmtrakV3Client:
    """
    Minimal Amtraker v3 API client for live trains and station metadata.
    No API key required.
    """
    def __init__(self, base_url: str = AMTRAK_V3_BASE_URL, timeout: int = 20, session: requests.Session | None = None):
        self.base_url = base_url.rstrip("/")
        self.timeout = timeout
        self._s = session or requests.Session()
        self._s.headers.update({"accept": "application/json"})

    # ---------- internal helper ----------

    def _get(self, path: str, params: dict | None = None) -> t.Any:
        url = f"{self.base_url}/{path.lstrip('/')}"
        r = self._s.get(url, params=params or {}, timeout=self.timeout)
        try:
            r.raise_for_status()
        except requests.HTTPError as e:
            # Surface JSON body (if any) to make debugging easier
            try:
                err = r.json()
                # v3 typically returns plain objects; on errors you may just see text
                details = err if isinstance(err, str) else "; ".join(map(str, err.values())) if isinstance(err, dict) else str(err)
            except Exception:
                details = r.text
            raise requests.HTTPError(f"{e} | Server said: {details}") from None
        return r.json()

    # ---------- trains ----------

    def get_all_trains(self) -> dict[str, list[dict]]:
        """All active trains. Dict keyed by train number -> list of train objects."""
        return self._get("/v3/trains")

    def get_train(self, train_id_or_num: str | int) -> dict[str, list[dict]]:
        """
        One train by number or by 'trainId' (e.g., '5-9' for train 5 that originated on the 9th).
        Returns a dict keyed by the train number.
        """
        return self._get(f"/v3/trains/{train_id_or_num}")

    def list_train_numbers(self) -> list[str]:
        """Convenience: list keys from get_all_trains()."""
        return list((self.get_all_trains() or {}).keys())

    # ---------- stations (metadata) ----------

    def get_all_stations(self) -> dict[str, dict]:
        """All station metadata keyed by station ID (e.g., 'CHI', 'BOS')."""
        return self._get("/v3/stations")

    def get_station(self, station_id: str) -> dict[str, dict]:
        """One station's metadata keyed by station ID."""
        return self._get(f"/v3/stations/{station_id}")

    # ---------- health ----------

    def get_stale_status(self) -> dict:
        """Feed health (avgLastUpdate, activeTrains, stale)."""
        return self._get("/v3/stale")

    # ---------- flatteners ----------

    @staticmethod
    def trains_dict_to_records(trains_by_num: dict[str, list[dict]]) -> list[dict]:
        """
        Flatten /v3/trains response to one row per train object.
        Fields vary a bit across releases; this is tolerant to missing keys.
        """
        rows: list[dict] = []
        for train_num, items in (trains_by_num or {}).items():
            for tdata in items or []:
                a = tdata or {}
                rows.append({
                    "trainNum": a.get("trainNum") or train_num,
                    "trainId": a.get("trainId"),
                    "routeName": a.get("routeName"),
                    "state": a.get("trainState") or a.get("state"),
                    "statusMsg": a.get("statusMsg"),
                    "lat": a.get("lat"),
                    "lon": a.get("lon"),
                    "heading": a.get("heading"),
                    "velocity": a.get("velocity"),
                    "lastValTS": a.get("lastValTS") or a.get("updatedAt"),
                    "eventCode": a.get("eventCode"),
                    "origCode": a.get("origCode"),
                    "destCode": a.get("destCode"),
                    # keep a count of stations array if present
                    "stations_len": len(a.get("stations", [])) if isinstance(a.get("stations"), list) else None,
                })
        return rows

    @staticmethod
    def stations_dict_to_records(stations_by_id: dict[str, dict]) -> list[dict]:
        """
        Flatten station metadata dict to simple rows.
        Keys differ by source; we preserve common ones when present.
        """
        out: list[dict] = []
        for sid, meta in (stations_by_id or {}).items():
            m = meta or {}
            out.append({
                "stationId": sid,
                "name": m.get("name") or m.get("stationName"),
                "city": m.get("city"),
                "state": m.get("state"),
                "tz": m.get("tz"),
                "lat": m.get("lat"),
                "lon": m.get("lon"),
            })
        return out

    # ---------- polling helpers ----------

    def poll_trains(
        self,
        train_ids_or_nums: list[str | int] | None = None,
        interval_sec: float = 10.0,
        iterations: int = 6,
        to_dataframe: bool = True,
        verbose: bool = True,
    ):
        """
        Poll specific trains by number or trainId; if None, polls all trains each tick.
        """
        snaps: list[dict] = []
        for i in range(iterations):
            if train_ids_or_nums:
                tick_dict: dict[str, list[dict]] = {}
                for tname in train_ids_or_nums:
                    d = self.get_train(tname) or {}
                    for k, v in d.items():
                        tick_dict.setdefault(k, []).extend(v or [])
            else:
                tick_dict = self.get_all_trains() or {}

            rows = self.trains_dict_to_records(tick_dict)
            snaps.append({"t_index": i, "ts": time.time(), "rows": rows})
            if verbose:
                msg = f"[{i+1}/{iterations}] fetched {len(rows)} train rows"
                if train_ids_or_nums:
                    msg += f" across {len(train_ids_or_nums)} target(s)"
                print(msg)
            if i < iterations - 1:
                time.sleep(interval_sec)

        if _HAS_PANDAS and to_dataframe:
            df_rows = []
            for s in snaps:
                for r in s["rows"]:
                    rr = dict(r)
                    rr["_t_index"] = s["t_index"]
                    rr["_ts"] = s["ts"]
                    df_rows.append(rr)
            return pd.DataFrame(df_rows)
        return snaps



In [5]:

client = AmtrakV3Client()

# 1) All trains (dict), and quick look as DataFrame:
all_trains = client.get_all_trains()
df = pd.DataFrame(AmtrakV3Client.trains_dict_to_records(all_trains)) if _HAS_PANDAS else all_trains
df.head() if _HAS_PANDAS else list(all_trains)[:5]

,trainNum,trainId,routeName,state,statusMsg,lat,lon,heading,velocity,lastValTS,eventCode,origCode,destCode,stations_len
0,1,None,Sunset Limited,Active,,31.772745,-106.470812,SW,0.000000,2025-08-19T12:57:00-06:00,ELP,NOL,LAX,22
1,2,None,Sunset Limited,Active,,30.122085,-94.425643,E,59.409279,2025-08-19T13:56:56-05:00,BMT,LAX,NOL,22
2,3,None,Southwest Chief,Active,,35.387648,-105.365107,W,45.676979,2025-08-19T12:56:58-06:00,LMY,CHI,LAX,32
3,4,None,Southwest Chief,Active,,39.610057,-93.111495,NE,88.899551,2025-08-19T13:56:57-05:00,LAP,LAX,CHI,32
4,4,None,Southwest Chief,Active,,35.497396,-106.221507,SE,63.224499,2025-08-19T12:47:57-06:00,LMY,LAX,CHI,32


In [ ]:

# -------------------------
# Example usage (uncomment to run)
# -------------------------
# client = AmtrakV3Client()
#
# # 1) All trains (dict), and quick look as DataFrame:
# all_trains = client.get_all_trains()
# df = pd.DataFrame(AmtrakV3Client.trains_dict_to_records(all_trains)) if _HAS_PANDAS else all_trains
# df.head() if _HAS_PANDAS else list(all_trains)[:5]
#
# # 2) One train by *number* (e.g., 48) or by *trainId* (e.g., "5-9"):
# t = client.get_train(48)           # by number
# # t = client.get_train("5-9")      # by trainId: train 5 originated on the 9th
#
# # 3) Station metadata:
# stations = client.get_all_stations()
# st_df = pd.DataFrame(AmtrakV3Client.stations_dict_to_records(stations)) if _HAS_PANDAS else stations
# st_df.head() if _HAS_PANDAS else list(stations)[:5]
#
# # 4) Feed health:
# client.get_stale_status()
#
# # 5) “Live-ish” polling of all trains for ~1 minute:
# polled = client.poll_trains(interval_sec=10, iterations=6, to_dataframe=True)
# polled.head() if _HAS_PANDAS else polled[:1]
